In [1]:
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from IPython.display import display
from ipywidgets import Dropdown, interact, fixed, Select, interactive, interact_manual
import ipywidgets as widgets
from matplotlib import patheffects
import matplotlib.pyplot as plt
from matplotlib import colors
from netCDF4 import Dataset
import numpy as np
from scipy import interpolate
from siphon.catalog import TDSCatalog
import xarray as xr
%matplotlib inline

In [2]:
year_array = np.arange(1979,2020,1)
month_array = np.arange(1,13,1)
day_array = np.arange(1,32,1)
levels_array = [1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,  775.,
        750.,  700.,  650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,
        250.,  225.,  200.,  175.,  150.,  125.,  100.,   70.,   50.,   30.,
         20.,   10.]

In [3]:
def open_dataset (year, filePrefix):
    """
    Open and return a netCDF Dataset object for a given date, channel, and image index
    of GOES-16 data from THREDDS.
    """

    server = 'http://thredds.atmos.albany.edu:8080/thredds/dodsC/CFSR/{}/{}.{}.0p5.anl.nc'
    dataset = server.format(year,filePrefix,year)
    print(dataset)
    ds = xr.open_dataset(dataset)
#    print(ds)
    return ds

In [4]:
def plot_CFSR(year,month,date,hour,field,level,proj):
    """
    Receives options from the interface and generates a plot.
    """
    
        
    if (field == 'Geopotential Height'):
        filePrefix = 'g'
        minVal = 0
        maxVal = 20000
        if (level >= 700):
            cinterval = 30
        elif (level >= 450):
            cinterval = 60
        elif (level >= 300):
            cinterval = 90
        else:
            cinterval = 120
        scale = 0
        addValue = 0
    elif (field == 'Temperature'):
        filePrefix = 't'
        minVal = -99
        maxVal = 51
        cinterval = 3
        scale = 0
        addValue = -273
    elif (field == 'Specific Humidity'):
        filePrefix = 'q'
    elif (field == 'Omega'):
        filePrefix = 'w'
    else:
        filePrefix = 'g'
    yearStr = str(year)
    monthStr = str(month).zfill(2)
    dayStr = str(date).zfill(2)
    dattim = '{}-{}-{} {}:00:00'
    ds = open_dataset(yearStr, filePrefix)

    lon = ds['lon']
    lat = ds['lat']
    z = ds[filePrefix].sel(lev=level).sel(time=dattim.format(yearStr,monthStr,dayStr,hour)) + addValue
        
    extent = False
    if (proj == 'PlateCarree'):
        projType = ccrs.PlateCarree()
        x = lon.values
        y = lat.values
    elif (proj) == 'Lambert US':
        extent = True
        lon1 = 265.
        lat1 = 25.
        slat = 25.
        lonW = -152.73
        lonE = -49.5
        latS = 12.140
        latN = 57.338
        x = lon.sel(lon=slice(lonW,lonE))
        y = lat.sel(lat=slice(latS,latN))
        z = z.sel(lat=slice(latS, latN), lon = slice(lonW,lonE))
        projType = ccrs.LambertConformal(central_longitude=lon1,
                             central_latitude=lat1,
                             standard_parallels=[slat])
#    elif (proj == 'Geostationary'):
#        lon1 = 265.
#        projType = ccrs.Geostationary(central_longitude=lon1)
    else: 
        projType = ccrs.PlateCarree()

#    print (y)
#    print(z)
    
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(1,1,1,projection=projType)
    if (extent):
        ax.set_extent ([lonW,lonE,latS,latN-5] )
    
    ax.coastlines(resolution='50m', color='black')
    ax.add_feature(cfeat.BORDERS, linewidths=2, edgecolor='black')
    state_boundaries = cfeat.NaturalEarthFeature(category='cultural',
                         name='admin_1_states_provinces_lines',
                         scale='50m', facecolor='none')
    ax.add_feature(state_boundaries, linestyle=':',linewidths=1,edgecolor='black')
    tl1 = str("CFSR " + str(level)+ ' ' + field)
    tl2 = str('Valid at: '+yearStr +monthStr + dayStr + " " + hour + "00 UTC")
    plt.title(tl1+'\n'+tl2,fontsize=16)
    cint = np.arange(minVal,maxVal,cinterval)
#Contour:
    CS = ax.contour(x,y,z.values,levels=cint,linewidths=1,colors='green',linestyles='solid',transform=ccrs.PlateCarree())
#Labels:
    ax.clabel(CS, inline=1, linewidths=3, fontsize=13,fmt='%.0f')


In [5]:
field = Select(
    options=['Geopotential Height', 'Temperature', 'Specific Humidity', 'Windspeed', 'Omega'],
    value = 'Geopotential Height',
    description='Parameter: ',
)

level = Dropdown(
    options=levels_array,
    value = 500,
    description='Pressure level: ',
)

year = Dropdown(
    options=year_array,
    value = 2018,
    description='Year'
)

month = Dropdown(
    options=month_array,
    value = 1,
    description='Month'
)

date = Dropdown(
    options=day_array,
    value = 1,
    description='Date'
)

hour = Dropdown(
    options=['00','06','12','18'],
    value = '00',
    description='Hour'
)

proj = Dropdown(
    options = ['PlateCarree', 'Lambert US'],
    value = 'PlateCarree',
    description = 'Map Projection'
)

w = interact_manual(plot_CFSR, year=year, month=month, date=date, hour=hour, 
         field=field, level=level, proj = proj)

interactive(children=(Dropdown(description='Year', index=39, options=(1979, 1980, 1981, 1982, 1983, 1984, 1985…